In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import random

import os

current_wd = str(os.getcwd())
parent_parent_parent = os.path.dirname(os.path.dirname(os.path.dirname(current_wd)))
os.chdir(parent_parent_parent)

from MyImports import *
hv.extension('bokeh')

os.chdir(current_wd)

seed = 68
random.seed(seed)
pyro.set_rng_seed(seed)
torch.manual_seed(seed);

In [ ]:
ground_truth = torch.tensor([[1.8], [1.7], [0.4], [0.3], [-1.8]], dtype=torch.float64)

In [ ]:
def update_xtest(model):
    min_range = model.kernel.module_1.weight.min().item() - 0.5
    max_range = model.kernel.module_1.weight.max().item() + 0.5
    
    XX = torch.meshgrid((x_con, torch.linspace(min_range, max_range, 100, dtype=torch.float64)), indexing='ij')
    X_test = torch.cat((XX[0].reshape(-1,1), XX[1].reshape(-1,1)), dim=1)
    
    return X_test

In [ ]:
def update_pseudo(model):
    model_cpy = PyroBO(X=torch.cat((model.X.T[0:1].T, model.kernel.module_1(model.X.T[1:].T)), dim=1).data,
                       y=model.y,
                       kernel=gp.kernels.RBF(input_dim=input_dim),
                       acq_fun=model.acq_fun)
    
    #model_cpy.kernel.lengthscale = pyro.nn.PyroSample(pyro.distributions.Beta(1,1))
    model_cpy.update()
    
    return model_cpy

In [ ]:
x_con = torch.linspace(0,5,100, dtype=torch.float64)
x_categ1 = torch.tensor([[1,0,0,0,0]], dtype=torch.float64)
x_categ2 = torch.tensor([[0,1,0,0,0]], dtype=torch.float64)
x_categ3 = torch.tensor([[0,0,1,0,0]], dtype=torch.float64)
x_categ4 = torch.tensor([[0,0,0,1,0]], dtype=torch.float64)
x_categ5 = torch.tensor([[0,0,0,0,1]], dtype=torch.float64)

X_categ = [x_categ1, x_categ2, x_categ3, x_categ4, x_categ5]

num_categ = len(X_categ)

In [ ]:
y_obj1 = Objective2D(x1=x_con, x2=ground_truth[0], noise=0.)
y_obj2 = Objective2D(x1=x_con, x2=ground_truth[1], noise=0.)
y_obj3 = Objective2D(x1=x_con, x2=ground_truth[2], noise=0.)
y_obj4 = Objective2D(x1=x_con, x2=ground_truth[3], noise=0.)
y_obj5 = Objective2D(x1=x_con, x2=ground_truth[4], noise=0.)

y_objs = [y_obj1, y_obj2, y_obj3, y_obj4, y_obj5]

In [ ]:
plot1 = hv.Curve((x_con, y_obj1), label='Objective')
plot2 = hv.Curve((x_con, y_obj2))
plot3 = hv.Curve((x_con, y_obj3))
plot4 = hv.Curve((x_con, y_obj4))
plot5 = hv.Curve((x_con, y_obj5))

plots = [plot1, plot2, plot3, plot4, plot5]

In [ ]:
def initialize_model(num_init=2):
    X_init = None
    for x_categ in X_categ:
        new_samples = torch.cat((torch.rand(num_init,1, dtype=torch.float64)*5,
                                 x_categ.repeat(num_init,1)), dim=1)
        
        if X_init is None:
            X_init = new_samples
        else:
            X_init = torch.cat((X_init, new_samples), dim=0)
            
    y_init = Objective2D(x1=X_init.T[0:1].T, x2=X_init.T[1:].T@ground_truth, noise=noise)
    
    mappings = OrderedDict({(0.): lambda x: x,
                            tuple(range(1, 1+num_categ)): torch.nn.Linear(num_categ, 1,
                                                                          bias=False,
                                                                          dtype=torch.float64)})
    kernel = TransformationKernel(input_dim = input_dim,
                                  trans_mappings = mappings)
    model = PyroBO(X=X_init,
                   y=y_init,
                   kernel=kernel,
                   acq_fun=ConfidenceBound(input_dim=input_dim,
                                           kappa=kappa,
                                           maximize=True))
    
    #model.kernel.lengthscale = pyro.nn.PyroSample(pyro.distributions.Beta(1,1))
    model.update()
    
    return model

In [ ]:
X_test_categ = torch.cat((x_con.view(-1,1), x_categ1.repeat(len(x_con),1)), dim=1)

for x_categ in X_categ[1:]:
    new_x = torch.cat((x_con.view(-1,1), x_categ.repeat(len(x_con), 1)), dim=1)
    X_test_categ = torch.cat((X_test_categ, new_x))

In [ ]:
my_X1 = torch.linspace(0,5,100)
my_X2 = torch.linspace(-2.5,2.5,100)
XX = torch.meshgrid((my_X1, my_X2), indexing='ij')
X = torch.cat((XX[0].reshape(-1,1), XX[1].reshape(-1,1)), dim=1)
y_obj = Objective2D(x1=XX[0].flatten(), x2=XX[1].flatten(), noise=0.)

In [ ]:
data = [(xi.item(), yi.item(), zi.item()) for xi, yi, zi in zip(X.T[0], X.T[1], y_obj)]

colors = ['green', 'red', 'blue', 'purple', 'pink']

categ1 = hv.Curve((x_con, (ground_truth[0]).repeat(100)), label='categ1').opts(color='green')
categ2 = hv.Curve((x_con, (ground_truth[1]).repeat(100)), label='categ2').opts(color='red')
categ3 = hv.Curve((x_con, (ground_truth[2]).repeat(100)), label='categ3').opts(color='blue')
categ4 = hv.Curve((x_con, (ground_truth[3]).repeat(100)), label='categ4').opts(color='purple')
categ5 = hv.Curve((x_con, ground_truth[4].repeat(100)), label='categ5').opts(color='pink')

obj_plot = hv.HeatMap(data).opts(cmap='RdBu_r')

In [ ]:
hv.extension('bokeh')

( obj_plot * categ1*categ2*categ3*categ4*categ5).opts(show_legend=True,
                                                      width=550, height=500, legend_position='left')

In [ ]:
def getHeatMap(init_model, y_obj, num_iter=2, params=None):

    hmap_dict = OrderedDict()
    
    for j in range(num_iter):
        
        if j != 0:
            y_next = Objective2D(x1=x_next.T[0:1].T, x2=x_next.T[1:].T@ground_truth, noise=noise)
            init_model.add_Observation(x_next, y_next)
        
        init_model.update()
        
        print('Iteration {}:\n noise: {:.2f}, l:{:.2f}, lambda: {:.2f}'.format(j, init_model.noise,
                                                                                init_model.kernel.lengthscale,
                                                                                init_model.kernel.variance))
        if params is not None:
            params['noise'].append(init_model.noise)
            params['lengthscale'].append(init_model.kernel.lengthscale)
            params['lambda'].append(init_model.kernel.variance)
            
        X_test = update_xtest(init_model)
        model_cpy = update_pseudo(init_model)
        
        data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0], X_test.T[1], model_cpy(X_test)[0].data)]
        map_plot = hv.HeatMap(data=data)#, vdims=hv.Dimension('z', range=(y_obj.min()-0.5, y_obj.max()+0.5)))
        map_plot = map_plot.opts(cmap='RdBu_r', width=480, colorbar=True, show_legend=False,
                                 title='Mean prediction', xlabel='continuous variable', ylabel='categorical variable')
        
        data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0], X_test.T[1], model_cpy(X_test)[1].data)]
        unc_plot = hv.HeatMap(data=data)
        unc_plot = unc_plot.opts(cmap='RdBu_r', width=380, colorbar=True,
                                 title='Prediction Uncertainty', xlabel='continuous variable', ylabel='categorical variable')

        for i in range(len(colors)):
            points = model_cpy.X[[(xi.T[1] == init_model.kernel.module_1.weight.T[i]).item() for xi in model_cpy.X]]
            points = points.data.numpy()
            new_plot = hv.Points((points.T[0], points.T[1]), label='categ{}'.format(i+1))
            new_plot= new_plot.opts(color=colors[i], size=5)
            map_plot = (map_plot * new_plot).opts(legend_position='left')
            unc_plot = (unc_plot * new_plot).opts(show_legend=False)
            
        data = [(x.item(), y.item(), z.item()) for x,y,z in zip(X_test.T[0],
                                                                X_test.T[1],
                                                                model_cpy.get_acq_values(X_test).data)]
        acq_plot = hv.HeatMap(data=data).opts(cmap='plasma', width=480, colorbar=True,
                                              title='Acquisition function', xlabel='continuous variable', ylabel='categorical variable')
        
        x_next = init_model.next_sample(X_test_categ).data
        x_next_trans = torch.cat((x_next.T[0:1], init_model.kernel.module_1(x_next.T[1:].T)), dim=1).data.numpy()
        print('next: ', x_next)
        next_plot = hv.Points(x_next_trans).opts(marker='diamond', color='white', size=7)
        acq_plot = acq_plot * next_plot
        
        hmap_dict[j] = (map_plot + unc_plot + acq_plot).opts(shared_axes=False).cols(2)
        
        
    return hmap_dict

In [ ]:
def simpleMeanPlot(model, x_con, X_categ):
    weights = model.kernel.module_1.weight
    model_cpy = update_pseudo(model)
    
    plots = None
    for idx in range(num_categ):
        
        x_categ = torch.cat((x_con.view(-1,1), X_categ[idx].repeat(len(x_con),1)),dim=1).data
        X = torch.cat((x_con.view(-1,1), weights.T[0].T.repeat(len(x_con),1)),dim=1).data
        
        mean_values = model(x_categ)[0].data.numpy()
        unc_values = model(x_categ)[1].data.numpy()
        acq_values = model.get_acq_values(x_categ).data.numpy()
        
        x_next = model_cpy.next_sample(X).data.numpy()
        
        indices = [(xi == weights.T[idx]).item() for xi in model_cpy.X.T[1:].T]
        
        new_plot = simple1D_Plot(X.T[0:1].T.numpy(),
                                 model_cpy.X.T[0:1].T[indices].data.numpy(),
                                 model_cpy.y[indices],
                                 mean_values,
                                 unc_values,
                                 acq_values,
                                 x_next.T[0:1].T,
                                 idx==0)[0].opts(width=480, height=400)
        if plots is None:
            plots = new_plot
        else:
            plots = plots + new_plot
            
    return plots.cols(1)

In [ ]:
import random

def set_seed(seed):
    torch.backends.cudnn.deterministic = True
    #torch.backends.cudnn.benchmark = False
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    pyro.set_rng_seed(seed)

## Extended BO - noise-free

In [ ]:
num_init = 3
num_iter = 30
noise = 0.
params = {'noise': [], 'lengthscale': [], 'lambda': []}
seed = 682
set_seed(seed)

input_dim = 2
kappa = 10.

bo_model = initialize_model(num_init)

In [ ]:
hmaps1 = hv.HoloMap(getHeatMap(bo_model, Objective2D, num_iter=num_iter, params=params),
                    kdims='Iterations')
hv.output(hmaps1.collate(), widget_location='top')

In [ ]:
hv.save(hmaps1.collate(), 'extendedCategorical.auto')

In [ ]:
hv.extension('plotly')

lengthscale_nf_plot = hv.Curve((torch.tensor(params['lengthscale']))).opts(height=200)
outputscale_nf_plot = hv.Curve((torch.tensor(params['lambda']))).opts(height=200)
noise_nf_plot = hv.Curve((torch.tensor(params['noise']))).opts(height=200)

(lengthscale_nf_plot + outputscale_nf_plot + noise_nf_plot).opts(shared_axes=False).cols(1)

## Extended BO - noisy

In [ ]:
set_seed(seed)
noise = 0.8
params = {'noise': [], 'lengthscale': [], 'lambda': []}

bo_model = initialize_model(num_init)

In [ ]:
X_test = update_xtest(bo_model)

In [ ]:
hv.extension('bokeh')

hmaps2 = hv.HoloMap(getHeatMap(bo_model, Objective2D, num_iter=num_iter, params=params),
                    kdims='Iterations')
hv.output(hmaps2.collate(), widget_location='top')

In [ ]:
hv.save(hmaps2.collate(), 'extendedCategorical_noisy.auto')

In [ ]:
hv.extension('plotly')

lengthscale_n_plot = hv.Curve((torch.tensor(params['lengthscale']))).opts(height=200)
outputscale_n_plot = hv.Curve((torch.tensor(params['lambda']))).opts(height=200)
noise_n_plot = hv.Curve((torch.tensor(params['noise']))).opts(height=200)

(lengthscale_n_plot + outputscale_n_plot + noise_n_plot).opts(shared_axes=False).cols(1)